In [234]:
import numpy as np

# С помощью библиотеки np реализовать модель с прямым проходом, состоящую из 3 полносвязных слоём с функциями потерь: ReLU, tanh, Softmax. Длины векторов на входе 256, на выходе 4, промежуточные: 64 и 16.

In [235]:
def ReLU(inputs):
    return np.maximum(0,inputs)

def tanh(inputs):
    return np.tanh(inputs)

def softMax(inputs):
    exponentsSum = np.sum(np.exp(inputs))
    f = np.vectorize(lambda x: np.exp(x)/exponentsSum)
    return f(inputs)

In [236]:
class Layer():

    def __init__(self, size, lossFuncs = []):
        self.weights = np.random.random(size)
        self.lossFuncs = lossFuncs

    def __call__(self, inputs):
        result = inputs.dot(self.weights)
        for f in self.lossFuncs:
            result = f(result)
        return result

In [237]:
np.random.seed(100)

inputs = np.random.randint(0, 255, size=(1, 256))
Neuron_1 = Layer((256, 64), [ReLU])
Neuron_2 = Layer((64, 16), [ReLU, tanh])
Neuron_3 = Layer((16, 4), [ReLU, softMax])

conveyorFirstTask = [Neuron_1, Neuron_2, Neuron_3]

result = inputs
for Layer in conveyorFirstTask:
    result = Layer(result)
result

array([[0.43849241, 0.25346715, 0.04080575, 0.26723469]])

# Реализовать модель с прямым проходом, состоящую из 2 свёрток с функциями активации ReLU и 2 функций MaxPool. Первый слой переводит из 3 каналов в 8, второй из 8 слоёв в 16. На вход подаётся изображение размера 19х19. (19х19x3 -> 18x18x8 -> 9x9x8 -> 8x8x16 -> 4x4x16)

In [238]:
class FoldLayer():

    def __init__(self, sizeIn, sizeOut, kernelSize, pad, step, activationFunc = ReLU):
        self.pad = pad
        self.step = step
        self.sIx, self.sIy, self.sIc = sizeIn
        self.sOx, self.sOy, self.sOc = sizeOut
        self.activatation = activationFunc

        self.weights = []
        for _ in range(sizeOut[2]):
            self.weights.append(np.random.random((kernelSize, kernelSize, self.sIc)))
        self.weights = np.array(self.weights)


    def __call__(self, inputs):
        return self.forward(inputs)

    def forward(self, inputs):
        ny, nx, nc = inputs.shape
    
        result = np.zeros((ny-1, nx-1, self.sOc)) 

        for ind, weight in enumerate(self.weights):
            applied = convolution(inputs, weight, step=self.step)
            result[:, :, ind] = applied[:, :]
        
        return self.activatation(result)

class MaxPoolLayer():

    def __init__(self, kernelSize, step):
        self.step = step
        self.kernelSize = kernelSize
    
    def __call__(self, inputs):
        return self.forward(inputs)

    def forward(self, inputs):
        result = maxPool(inputs, self.kernelSize, step=self.step)
        return result

def maxPool(x, kernelSize : int, step):
    inX, inY, inC = x.shape
    maxPoolResult = np.zeros((inX//kernelSize, inY//kernelSize, inC))

    for channel in range(inC):
        for v_shift in range(0, inX - kernelSize + 1, step):
            for h_shift in range(0, inY - kernelSize + 1, step):
                maxPoolResult[v_shift//kernelSize][h_shift//kernelSize][channel] = np.max(x[v_shift:v_shift+kernelSize,h_shift:h_shift+kernelSize,channel])
    return maxPoolResult

def pad(matr, pad1, pad2=None):
    if not pad2:
        pad2 = pad1
    x, y, c = matr.shape
    resMatr = np.zeros((x+pad1+pad2, y+pad1+pad2, c))
    resMatr[pad1:-pad2, pad1:-pad2] = matr
    return resMatr

def convolution(x, kernel, step):
    if x.shape[2] != kernel.shape[2]:
        raise ValueError("Shapes must be same")
    
    inX, inY, inC = x.shape
    kX, kY, kC = kernel.shape

    convResult = np.zeros((inX-1, inY-1, x.shape[2]))
    for channel in range(inC):
        for v_shift in range(0, inX - kX + 1, step):
            for h_shift in range(0, inY - kY + 1, step):
                convResult[v_shift][h_shift][channel] = np.sum(
                    x[v_shift:v_shift+kX,h_shift:h_shift+kY,channel] * kernel[:,:,channel]
                )
    convResult = np.sum(convResult, axis=2)
    return (convResult)


In [239]:
np.random.seed(100)

inputs = np.random.randint(0, 255, size=(19,19,3))

fst = FoldLayer((19,19,3), (18,18,8), kernelSize=3, pad=0, step=1)
snd = MaxPoolLayer(2, 2)
thrd = FoldLayer((9,9,8), (8,8,16), 3, 0, 1)
frt = MaxPoolLayer(2, 2)


conveyorSecondTask = [fst, snd, thrd, frt]
data = inputs
print(data.shape)
for layer in conveyorSecondTask:
    data = layer(data)
    print(data.shape)

(19, 19, 3)
(18, 18, 8)
(9, 9, 8)
(8, 8, 16)
(4, 4, 16)


# Объединить сети из п.2 и п.1. На выход изображение размера 19х19, на выходе вектор из 4 элементов

In [240]:
data = np.random.randint(0, 255, size=(19,19,3))

conveyorThirdTask = conveyorSecondTask + [np.ravel] + conveyorFirstTask

for layer in conveyorThirdTask:
    print(data.shape)
    data = layer(data)
print(data)

(19, 19, 3)
(18, 18, 8)
(9, 9, 8)
(8, 8, 16)
(4, 4, 16)
(256,)
(64,)
(16,)
[0.43849241 0.25346715 0.04080575 0.26723469]
